# Запуск Ollama в Google Colab через VDS

## Настройка секретов

Перед запуском добавьте следующие секреты в Google Colab:
1. `tunnel_key` - приватный SSH ключ от VDS сервера
2. `vds_host` - IP адрес вашего VDS сервера

Для добавления секретов:
1. Нажмите на иконку ключа в левой панели
2. Нажмите "Add new secret"
3. Добавьте секреты с указанными выше именами

## Настройка среды

In [ ]:
from google.colab import userdata
import os
import subprocess
import threading
import time

# Проверка наличия секретов
required_secrets = ['tunnel_key', 'vds_host']
missing_secrets = []

for secret in required_secrets:
    try:
        if not userdata.get(secret):
            missing_secrets.append(secret)
    except Exception:
        missing_secrets.append(secret)

if missing_secrets:
    raise ValueError(f"Missing required secrets: {', '.join(missing_secrets)}")

# Создание SSH ключа
os.makedirs(os.path.expanduser('~/.ssh'), exist_ok=True)
key_path = os.path.expanduser('~/.ssh/tunnel_key')

with open(key_path, 'w') as f:
    f.write(userdata.get('tunnel_key'))

os.chmod(key_path, 0o600)

# Получение адреса VDS
VDS_HOST = userdata.get('vds_host')
VDS_USER = 'tunnel'

print("✅ Секреты успешно загружены")

## Установка Ollama и необходимых пакетов

In [ ]:
# Установка Ollama
!curl https://ollama.ai/install.sh | sh

# Установка CUDA драйверов
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

# Настройка переменных окружения
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

## Вспомогательные функции

In [ ]:
def run_and_print_output(process):
    """Вспомогательная функция для вывода процесса"""
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

def check_tunnel():
    """Проверка SSH туннеля"""
    try:
        result = subprocess.run(
            ['ssh', '-i', key_path, '-o', 'StrictHostKeyChecking=no',
             f'{VDS_USER}@{VDS_HOST}', 'echo', 'test'],
            capture_output=True,
            text=True
        )
        return result.returncode == 0
    except Exception:
        return False

## Запуск Ollama и создание туннеля

In [ ]:
# Запуск Ollama
print('🚀 Запуск Ollama...')
ollama_serve = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Запуск потока для вывода Ollama
serve_thread = threading.Thread(target=run_and_print_output, args=(ollama_serve,))
serve_thread.daemon = True
serve_thread.start()

# Ждем запуска Ollama
time.sleep(5)

# Создание SSH туннеля
print('🔄 Создание SSH туннеля...')
ssh_command = f"ssh -N -R 11434:localhost:11434 -o StrictHostKeyChecking=no -i {key_path} {VDS_USER}@{VDS_HOST}"
tunnel = subprocess.Popen(ssh_command.split())

# Проверка туннеля
time.sleep(3)
if check_tunnel():
    print(f'✅ Туннель создан успешно!')
    print(f'\n🌐 Ollama доступен по адресу: http://{VDS_HOST}')
else:
    print('❌ Ошибка создания туннеля')
    ollama_serve.terminate()
    tunnel.terminate()
    raise Exception("Failed to create tunnel")

## Загрузка модели

In [ ]:
# Загрузка модели
print('📥 Загрузка модели qwen2.5-coder:32b...')
pull_process = subprocess.Popen(
    ['ollama', 'pull', 'qwen2.5-coder:32b'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Вывод процесса загрузки
while True:
    output = pull_process.stdout.readline()
    if output == '' and pull_process.poll() is not None:
        break
    if output:
        print(output.strip())

print('\n✅ Настройка завершена!')
print(f'🌐 Ollama доступен по адресу: http://{VDS_HOST}')

## Поддержание работы сервера

Выполните эту ячейку, чтобы сервер продолжал работать. Нажмите Ctrl+C для остановки.

In [ ]:
try:
    while True:
        if not check_tunnel():
            print('🔄 Переподключение туннеля...')
            tunnel.terminate()
            tunnel = subprocess.Popen(ssh_command.split())
            time.sleep(3)
        time.sleep(30)
except KeyboardInterrupt:
    print('\n⏹️ Остановка сервера...')
    ollama_serve.terminate()
    tunnel.terminate()
    print('✅ Сервер остановлен')